In [1]:
import nlp
import warnings
import tokenizers
import transformers

from sklearn.metrics import accuracy_score

In [2]:
warnings.filterwarnings('ignore')

### Part 1: Train base BERT tokenizer

In [3]:
train, test = nlp.load_dataset("emo", split = ["train", "test"])

In [4]:
train_text = " ".join([i["text"] for i in train])
test_text = " ".join([i["text"] for i in test])

In [5]:
with open('../data/train.txt', 'w') as f:
    f.write(train_text)
with open('../data/test.txt', 'w') as f:
    f.write(test_text)

In [6]:
tokenizer = tokenizers.BertWordPieceTokenizer()

In [7]:
vocab_size = 5000

tokenizer.train(files = ['../data/train.txt', '../data/test.txt'], \
                vocab_size = vocab_size, min_frequency = 50)

In [8]:
tokenizer.save_model('../tokenizers/emo-mobilebert/')

['../tokenizers/emo-mobilebert/vocab.txt']

In [9]:
tokenizer = transformers.MobileBertTokenizerFast.from_pretrained('../tokenizers/emo-mobilebert/')

### Part 2: Instantiate MobileBERT Model and reset params

In [10]:
config = transformers.MobileBertConfig(vocab_size = len(tokenizer.get_vocab()))

In [11]:
config.num_labels = 4

In [12]:
config.max_length = 128

In [13]:
id2label = {}
for i in range(config.num_labels):
    id2label[i] = train.features["label"].int2str(i)
id2label

{0: 'others', 1: 'happy', 2: 'sad', 3: 'angry'}

In [14]:
config.id2label = id2label

In [15]:
config.label2id = {v:k for k,v in id2label.items()}

In [16]:
model = transformers.MobileBertForSequenceClassification(config)

In [17]:
model.config

MobileBertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": true,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "id2label": {
    "0": "others",
    "1": "happy",
    "2": "sad",
    "3": "angry"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "label2id": {
    "angry": 3,
    "happy": 1,
    "others": 0,
    "sad": 2
  },
  "layer_norm_eps": 1e-12,
  "max_length": 128,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "use_bottleneck": true,
  "use_bottleneck_attention": false,
  "vocab_size": 2016
}

### Part 3: Training

In [18]:
## No max len defined as all sentences are not too long
def tokenize(batch):
    return tokenizer(batch['text'], padding = True)

In [19]:
train_dataset = train.map(tokenize, batched = True, batch_size = len(train))
test_dataset = test.map(tokenize, batched = True, batch_size = len(train))
train_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])

In [20]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }

In [21]:
training_args = transformers.TrainingArguments(
    output_dir = './results',
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,
    weight_decay = 0.01,
    evaluate_during_training = True,
    logging_dir = './logs',
)

trainer = transformers.Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [ ]:
trainer.train()